In [7]:
import sys
sys.path.append("/home/aayushjoshi/ML/MLenv/lib/python3.11/site-packages")

In [8]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

In [9]:
df = pd.read_csv('/home/aayushjoshi/ML/Projects/emotions/dataset/dataset_val.csv')
df.describe()

,ID,Text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
count,5426,5426,5426,5426,5426,5426,5426,5426,5426,5426,...,5426,5426,5426,5426,5426,5426,5426,5426,5426,5426
unique,5426,5423,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
top,edgurhb,I like it!,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,1,3,4938,5123,5231,5123,5029,5273,5274,5178,...,5174,5405,5217,5411,5299,5408,5358,5283,5297,3660


In [10]:
labels = df.columns[2:]
labels

Index(['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
       'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
       'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
       'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
       'relief', 'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')

In [11]:
reduced_labels = ['joy', 'sadness', 'nervousness', 'anger', 'disgust', 'optimism', 'grief', 'neutral']

In [12]:
num_classes = len(reduced_labels)
num_classes

8

In [13]:
for label in labels:
    if label not in reduced_labels:
        df = df.drop(label, axis=1)

In [14]:
df.iloc[:, 2:] = df.iloc[:, 2:].astype(int)

In [15]:
df.columns

Index(['ID', 'Text', 'anger', 'disgust', 'grief', 'joy', 'nervousness',
       'optimism', 'sadness', 'neutral'],
      dtype='object')

In [16]:
def accumulate(row):
    return list(row.values)

df['label'] = df.iloc[:, 2:].apply(accumulate, axis=1)

In [17]:
df = df.drop(reduced_labels, axis=1)

In [18]:
max(len(text) for text in df['Text'])

187

In [19]:
import string

def remove_numbers(text):
    return ''.join(char for char in text if not char.isdigit())

def remove_special_characters(text):
    return ''.join(char for char in text if char not in string.punctuation)

for i in range(len(df['Text'])):
    df['Text'][i] = remove_special_characters(df['Text'][i])
    df['Text'][i] = remove_numbers(df['Text'][i])
    df['Text'][i] = df['Text'][i].lower()


In [20]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_text(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,  # change it when optimizing the model
        return_token_type_ids=True,
        pad_to_max_length=True,
        return_attention_mask=True,
        truncation=True
    )

In [21]:
import tensorflow as tf

def convert_to_tf_dataset(texts, labels, batch_size=32):
    input_ids = []
    attention_masks = []
    token_type_ids = []

    for text in texts:
        tokens = tokenize_text(text)
        input_ids.append(tokens['input_ids'])
        attention_masks.append(tokens['attention_mask'])
        token_type_ids.append(tokens['token_type_ids'])

    input_ids = tf.convert_to_tensor(input_ids)
    attention_masks = tf.convert_to_tensor(attention_masks)
    token_type_ids = tf.convert_to_tensor(token_type_ids)
    labels = tf.convert_to_tensor(labels)

    dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': input_ids, 'attention_mask': attention_masks, 'token_type_ids': token_type_ids}, labels))

    return dataset.batch(batch_size)

texts = list(df['Text'])
labels = list(df['label'])
batch_size = 8 # another hyperparameter we will fine tune later
train_dataset = convert_to_tf_dataset(texts, labels, batch_size=batch_size)

2024-01-07 15:46:23.343126: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 15:46:23.343479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 15:46:23.435683: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-07 15:46:23.640824: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 15:46:25.299163: W tensorflow/compiler/tf2

In [22]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model.fit(train_dataset, epochs=10)

Epoch 1/10
679/679 [==============================] - 4787s 7s/step - loss: 0.6312 - accuracy: 0.3192
Epoch 2/10
679/679 [==============================] - 4798s 7s/step - loss: 0.5490 - accuracy: 0.3233
Epoch 3/10
679/679 [==============================] - 4808s 7s/step - loss: 0.5207 - accuracy: 0.3212
Epoch 4/10
679/679 [==============================] - 4783s 7s/step - loss: 0.5130 - accuracy: 0.3216
Epoch 5/10
679/679 [==============================] - 4762s 7s/step - loss: 0.5023 - accuracy: 0.3199
Epoch 6/10
679/679 [==============================] - 4719s 7s/step - loss: 0.4957 - accuracy: 0.3209
Epoch 7/10
679/679 [==============================] - 4768s 7s/step - loss: 0.4774 - accuracy: 0.3207
Epoch 8/10
679/679 [==============================] - 4810s 7s/step - loss: 0.4774 - accuracy: 0.3212
Epoch 9/10
679/679 [==============================] - 4811s 7s/step - loss: 0.4647 - accuracy: 0.3225
Epoch 10/10
679/679 [==============================] - 4808s 7s/step - loss: 0.499

In [25]:
model.save('berta')

INFO:tensorflow:Assets written to: berta/assets


INFO:tensorflow:Assets written to: berta/assets
